In [1]:
import pandas as pd
import numpy as np
import ssp_eq_mws_sfr as sspeq

In [2]:
np.set_printoptions(precision=3)

In [3]:
path = r"C:\Users\misha\Desktop\python_exercises\#6 module for Steady-State Productivity Equations for a Multiple-Wells System in Sector Fault Reservoirs\input_data_template.xlsx"
file = pd.read_excel(path, header=None)

In [4]:
file

0     1     2    3    4      5    6     7
0    START   NaN   NaN  NaN  NaN    NaN  NaN   NaN
1     RESV   NaN   NaN  NaN  NaN    NaN  NaN   NaN
2      P_e   R_e   PHI    H  K_r    K_z   mu     B
3       18  2500    50   20  0.1  0.025    5  1.25
4     WELL   NaN   NaN  NaN  NaN    NaN  NaN   NaN
..     ...   ...   ...  ...  ...    ...  ...   ...
154      5   4.8   200   25  0.1  12.03  NaN   NaN
155      6   8.7  1500   30  0.1  32.59  NaN   NaN
156      7   5.8   600   35  0.1  34.03  NaN   NaN
157      8   9.3  2000   40  0.1  52.45  NaN   NaN
158    END   NaN   NaN  NaN  NaN    NaN  NaN   NaN

[159 rows x 8 columns]

In [5]:
def get_case_bound(dataframe):
    start = dataframe.index[dataframe[0] == 'START'].to_list()
    end = dataframe.index[dataframe[0] == 'END'].to_list()
    return start, end

print(*get_case_bound(file))

[0, 16, 32, 48, 64, 80, 96, 112, 128, 144] [14, 30, 46, 62, 78, 94, 110, 126, 142, 158]


In [6]:
start_i, end_i = get_case_bound(file)

print(start_i, end_i)

[0, 16, 32, 48, 64, 80, 96, 112, 128, 144] [14, 30, 46, 62, 78, 94, 110, 126, 142, 158]


In [7]:
case = file.iloc[[_ for _ in range(start_i[0] + 1, end_i[0])]].reset_index(drop=True)
case

0     1     2      3    4      5    6     7
0    RESV   NaN   NaN    NaN  NaN    NaN  NaN   NaN
1     P_e   R_e   PHI      H  K_r    K_z   mu     B
2      18  2500    50     20  0.1  0.025    5  1.25
3    WELL   NaN   NaN    NaN  NaN    NaN  NaN   NaN
4   index  P_wf   R_i  Phi_i  r_w   SKIN  NaN   NaN
5       1   5.1   300      5  0.1      5  NaN   NaN
6       2   5.2   400     10  0.1      5  NaN   NaN
7       3   6.4   800     15  0.1      5  NaN   NaN
8       4   5.5  1000     20  0.1      5  NaN   NaN
9       5   4.8   200     25  0.1      5  NaN   NaN
10      6   8.7  1500     30  0.1      5  NaN   NaN
11      7   5.8   600     35  0.1      5  NaN   NaN
12      8   9.3  2000     40  0.1      5  NaN   NaN

In [8]:
def split_properties(dataframe):
    # boundaries
    resv_bound1 = int(dataframe.index[dataframe[0] == 'RESV'].to_list()[0])
    resv_bound2 = int(dataframe.index[dataframe[0] == 'WELL'].to_list()[0])
    # slicing
    resv_prop = dataframe.iloc[[_ for _ in range(resv_bound1 + 1, resv_bound2)]].reset_index(drop=True)
    well_prop = dataframe.iloc[resv_bound2+1:].reset_index(drop=True)
    # finising
    
    resv_prop.columns = resv_prop.iloc[0]
    resv_prop = resv_prop[1:]
    
    mode_marker = well_prop.iloc[0].to_list()[5].upper()
    
    well_prop.columns = well_prop.iloc[0]
    well_prop = well_prop[1:].dropna(axis=1).drop(columns='index')

    return resv_prop.values.tolist(), well_prop.values.tolist(), mode_marker

In [9]:
a, b, marker = split_properties(case)
print(a)
print(b)
print(marker)

[[18, 2500, 50, 20, 0.1, 0.025, 5, 1.25]]
[[5.1, 300, 5, 0.1, 5], [5.2, 400, 10, 0.1, 5], [6.4, 800, 15, 0.1, 5], [5.5, 1000, 20, 0.1, 5], [4.8, 200, 25, 0.1, 5], [8.7, 1500, 30, 0.1, 5], [5.8, 600, 35, 0.1, 5], [9.3, 2000, 40, 0.1, 5]]
SKIN


In [46]:
start_i, end_i = get_case_bound(file)

output_file = pd.DataFrame()

for i in range(len(start_i)):
    case_i = file.iloc[[_ for _ in range(start_i[i] + 1, end_i[i])]].reset_index(drop=True)
    
    resv_i, wells_i, mode = split_properties(case_i)
    if mode == 'SKIN':
        q_sum, q = sspeq.find_q(wells_i, *resv_i)
        print(f'CASE {i}\t{mode} is known\nQ_sum : {q_sum: .3f}\nQ_vec : {[round(_, 2) for _ in q]}')
        well_numeration = [f'well {i+1}' for i in range(len(q))] + ['SUM']
        q_vec = q + [q_sum]
        solution = pd.concat([pd.DataFrame([f'CASE {i}','Q (rate)']), pd.DataFrame([well_numeration, q_vec])])
    elif mode == 'Q':
        s_vec = sspeq.find_s(wells_i, *resv_i)
        print(f'CASE {i}\t{mode} is known\nS_vec : {[round(_, 2) for _ in s_vec]}')
        well_numeration = [f'well {i+1}' for i in range(len(s_vec))]
        solution = pd.concat([pd.DataFrame([f'CASE {i}','SKIN']), pd.DataFrame([well_numeration, s_vec])])
    else:
        pass
        
    output_file = pd.concat([output_file, solution])
                              
output_file

CASE 0	SKIN is known
Q_sum :  237.299
Q_vec : [13.4, 18.0, 23.92, 49.74, 16.66, 36.95, 22.5, 56.13]
CASE 1	SKIN is known
Q_sum :  270.460
Q_vec : [9.91, 16.57, 21.57, 64.34, 15.61, 43.56, 21.45, 77.45]
CASE 2	SKIN is known
Q_sum :  214.795
Q_vec : [14.99, 18.47, 23.5, 41.8, 17.26, 31.96, 22.13, 44.68]
CASE 3	SKIN is known
Q_sum :  310.672
Q_vec : [12.78, 19.93, 25.57, 68.92, 20.2, 49.47, 28.04, 85.76]
CASE 4	SKIN is known
Q_sum :  401.769
Q_vec : [21.79, 29.32, 34.28, 78.1, 36.32, 59.41, 44.01, 98.54]
CASE 5	SKIN is known
Q_sum :  464.118
Q_vec : [29.55, 36.47, 39.42, 83.17, 51.83, 64.28, 54.4, 104.99]
CASE 6	SKIN is known
Q_sum :  213.569
Q_vec : [12.06, 16.2, 21.53, 44.77, 14.99, 33.25, 20.25, 50.52]
CASE 7	SKIN is known
Q_sum :  237.299
Q_vec : [13.4, 18.0, 23.92, 49.74, 16.66, 36.95, 22.5, 56.13]
CASE 8	SKIN is known
Q_sum :  261.029
Q_vec : [14.74, 19.8, 26.31, 54.72, 18.33, 40.64, 24.75, 61.74]
CASE 9	Q is known
S_vec : [-1.3, 6.7, -1.53, 6.95, 4.83, 6.18, -1.38, 5.83]


0          1          2          3          4          5  \
0     CASE 0        NaN        NaN        NaN        NaN        NaN   
1   Q (rate)        NaN        NaN        NaN        NaN        NaN   
0     well 1     well 2     well 3     well 4     well 5     well 6   
1  13.399596  18.002336  23.922119  49.744796  16.660164  36.945545   
0     CASE 1        NaN        NaN        NaN        NaN        NaN   
1   Q (rate)        NaN        NaN        NaN        NaN        NaN   
0     well 1     well 2     well 3     well 4     well 5     well 6   
1   9.912616  16.568626  21.567875  64.337711    15.6118  43.559142   
0     CASE 2        NaN        NaN        NaN        NaN        NaN   
1   Q (rate)        NaN        NaN        NaN        NaN        NaN   
0     well 1     well 2     well 3     well 4     well 5     well 6   
1  14.991129  18.473136  23.500631  41.802286  17.258362   31.95797   
0     CASE 3        NaN        NaN        NaN        NaN        NaN   
1   Q (rate)        NaN        NaN        NaN        NaN        NaN   
0     well 1     well 2     well 3     well 4     well 5     well 6   
1  12.781524  19.929616  25.571501  68.923488  20.198981  49.473594   
0     CASE 4        NaN        NaN        NaN        NaN        NaN   
1   Q (rate)        NaN        NaN        NaN        NaN        NaN   
0     well 1     well 2     well 3     well 4     well 5     well 6   
1  21.788712  29.316981  34.279833  78.103358  36.315611  59.414981   
0     CASE 5        NaN        NaN        NaN        NaN        NaN   
1   Q (rate)        NaN        NaN        NaN        NaN        NaN   
0     well 1     well 2     well 3     well 4     well 5     well 6   
1    29.5516  36.466868  39.420942  83.168715   51.82947  64.281271   
0     CASE 6        NaN        NaN        NaN        NaN        NaN   
1   Q (rate)        NaN        NaN        NaN        NaN        NaN   
0     well 1     well 2     well 3     well 4     well 5     well 6   
1  12.059637  16.202103  21.529907  44.770316  14.994148   33.25099   
0     CASE 7        NaN        NaN        NaN        NaN        NaN   
1   Q (rate)        NaN        NaN        NaN        NaN        NaN   
0     well 1     well 2     well 3     well 4     well 5     well 6   
1  13.399596  18.002336  23.922119  49.744796  16.660164  36.945545   
0     CASE 8        NaN        NaN        NaN        NaN        NaN   
1   Q (rate)        NaN        NaN        NaN        NaN        NaN   
0     well 1     well 2     well 3     well 4     well 5     well 6   
1  14.739556   19.80257   26.31433  54.719275   18.32618  40.640099   
0     CASE 9        NaN        NaN        NaN        NaN        NaN   
1       SKIN        NaN        NaN        NaN        NaN        NaN   
0     well 1     well 2     well 3     well 4     well 5     well 6   
1  -1.301974   6.696889  -1.533469    6.94821   4.825143   6.180877   

           6           7           8  
0        NaN         NaN         NaN  
1        NaN         NaN         NaN  
0     well 7      well 8         SUM  
1  22.496333   56.127887  237.298776  
0        NaN         NaN         NaN  
1        NaN         NaN         NaN  
0     well 7      well 8         SUM  
1  21.449566   77.452999  270.460337  
0        NaN         NaN         NaN  
1        NaN         NaN         NaN  
0     well 7      well 8         SUM  
1  22.130016    44.68127  214.794801  
0        NaN         NaN         NaN  
1        NaN         NaN         NaN  
0     well 7      well 8         SUM  
1  28.037174   85.755714  310.671592  
0        NaN         NaN         NaN  
1        NaN         NaN         NaN  
0     well 7      well 8         SUM  
1  44.012227   98.537463  401.769167  
0        NaN         NaN         NaN  
1        NaN         NaN         NaN  
0     well 7      well 8         SUM  
1  54.404158  104.994846  464.117871  
0        NaN         NaN         NaN  
1        NaN         NaN         NaN  
0     well 7      well 8         SUM  
1    20

In [30]:
pd.concat([pd.DataFrame(['SKIN']), pd.DataFrame([[f'well {i+1}' for i in range(len(s_vec))], s_vec])])

0         1         2        3         4         5         6  \
0      SKIN       NaN       NaN      NaN       NaN       NaN       NaN   
0    well 1    well 2    well 3   well 4    well 5    well 6    well 7   
1 -1.301974  6.696889 -1.533469  6.94821  4.825143  6.180877 -1.384369   

          7  
0       NaN  
0    well 8  
1  5.825826

In [25]:
pd.concat([pd.DataFrame(['SKIN']), pd.DataFrame([s_vec], columns=[f'well {i+1}' for i in range(len(s_vec))])], axis=0)

0    well 1    well 2    well 3   well 4    well 5    well 6    well 7  \
0  SKIN       NaN       NaN       NaN      NaN       NaN       NaN       NaN   
0   NaN -1.301974  6.696889 -1.533469  6.94821  4.825143  6.180877 -1.384369   

     well 8  
0       NaN  
0  5.825826

In [48]:
output_file.to_excel('output_solution.xlsx', index=False, header=False)

In [53]:
string_i = 'hui\nt'
print(string_i)

string_i = repr(string_i).replace("'", '')
print(string_i)

hui
t
hui\nt


In [56]:
pd.read_excel()

TypeError: read_excel() missing 1 required positional argument: 'io'

In [57]:
pd.DataFrame([[5.1, 300, 5, 0.1, 5], [5.2, 400, 10, 0.1, 5], [6.4, 800, 15, 0.1, 5], [5.5, 1000, 20, 0.1, 5], [4.8, 200, 25, 0.1, 5], [8.7, 1500, 30, 0.1, 5], [5.8, 600, 35, 0.1, 5], [9.3, 2000, 40, 0.1, 5]])

0     1   2    3  4
0  5.1   300   5  0.1  5
1  5.2   400  10  0.1  5
2  6.4   800  15  0.1  5
3  5.5  1000  20  0.1  5
4  4.8   200  25  0.1  5
5  8.7  1500  30  0.1  5
6  5.8   600  35  0.1  5
7  9.3  2000  40  0.1  5

In [59]:
test = ((5.1, 300.0, 5.0, 0.1, 5.0), (5.2, 400.0, 10.0, 0.1, 5.0), (6.4, 800.0, 15.0, 0.1, 5.0), (5.5, 1000.0, 20.0, 0.1, 5.0), (4.8, 200.0, 25.0, 0.1, 5.0), (8.7, 1500.0, 30.0, 0.1, 5.0), (5.8, 600.0, 35.0, 0.1, 5.0), (9.3, 2000.0, 40.0, 0.1, 5.0))
print(test)

((5.1, 300.0, 5.0, 0.1, 5.0), (5.2, 400.0, 10.0, 0.1, 5.0), (6.4, 800.0, 15.0, 0.1, 5.0), (5.5, 1000.0, 20.0, 0.1, 5.0), (4.8, 200.0, 25.0, 0.1, 5.0), (8.7, 1500.0, 30.0, 0.1, 5.0), (5.8, 600.0, 35.0, 0.1, 5.0), (9.3, 2000.0, 40.0, 0.1, 5.0))


In [66]:
input_save = pd.concat([pd.DataFrame(['CASE MANUAL', 'INPUT']), pd.DataFrame([list(i) for i in test])])

input_save

0       1     2    3    4
0  CASE MANUAL     NaN   NaN  NaN  NaN
1        INPUT     NaN   NaN  NaN  NaN
0          5.1   300.0   5.0  0.1  5.0
1          5.2   400.0  10.0  0.1  5.0
2          6.4   800.0  15.0  0.1  5.0
3          5.5  1000.0  20.0  0.1  5.0
4          4.8   200.0  25.0  0.1  5.0
5          8.7  1500.0  30.0  0.1  5.0
6          5.8   600.0  35.0  0.1  5.0
7          9.3  2000.0  40.0  0.1  5.0

In [67]:
well_input_df = pd.DataFrame([['p_wf', 'r_i', 'phi_i', 'r_w', 'skin'], hui])

NameError: name 'hui' is not defined